In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [14]:
import pandas as pd
import itertools
from tqdm import tqdm_notebook

In [3]:
cd ..

/Share/home/shibinbin/projects/exRNA


In [5]:
with open('metadata/sample_ids.txt', 'r') as f:
    sample_ids = f.read().split()

rna_types = ['lncRNA', 'miRNA', 'mRNA', 'piRNA', 'snoRNA',
    'snRNA', 'srpRNA', 'tRNA', 'tucpRNA', 'vaultRNA', 'Y_RNA']

piranha_methods = ['ZeroTruncatedNegativeBinomial', 'ZeroTruncatedNegativeBinomialRegression',
    'ZeroTruncatedPoisson', 'ZeroTruncatedPoissonRegression']

In [13]:
peaks = []
for sample_id, method in itertools.product(sample_ids, piranha_methods):
    df = pd.read_table('output/call_peak_piranha/{method}/{sample_id}.bed'.format(**locals()), 
                       header=None, sep='\t')
    if df.shape[1] < 6:
        continue
    df = df.iloc[:, :6].copy()
    colnames = ['rna_id', 'start', 'end', 'name', 'rpkm', 'strand']
    df.columns = colnames
    #if df.shape[1] == 8:
    #    colnames.append('coef')
    #df.columns = colnames
    df['sample_id'] = sample_id
    df['method'] = method
    peaks.append(df)
peaks = pd.concat(peaks, axis=0)
peaks.head()

,rna_id,start,end,name,rpkm,strand,sample_id,method
0,ENST00000313495.6,2070,2190,X,60.0,+,10515058-A,ZeroTruncatedNegativeBinomial
1,ENST00000313495.6,2370,2400,X,40.0,+,10515058-A,ZeroTruncatedNegativeBinomial
2,ENST00000313495.6,2490,2550,X,84.0,+,10515058-A,ZeroTruncatedNegativeBinomial
3,ENST00000395900.1,2490,2520,X,38.0,+,10515058-A,ZeroTruncatedNegativeBinomial
4,ENST00000592946.1,2850,2880,X,47.0,+,10515058-A,ZeroTruncatedNegativeBinomial


In [16]:
peaks_RN7SL1 = peaks.query('rna_id == "ENST00000618786.1"')
peaks_RN7SL1.head()

,rna_id,start,end,name,rpkm,strand,sample_id,method
1625,ENST00000618786.1,0,240,X,70.125,+,10515058-A,ZeroTruncatedNegativeBinomial
2966,ENST00000618786.1,240,299,X,332.500,+,10515058-A,ZeroTruncatedNegativeBinomial
1199,ENST00000618786.1,240,299,X,332.500,+,10515058-A,ZeroTruncatedNegativeBinomialRegression
115755,ENST00000618786.1,0,240,X,70.125,+,10515058-A,ZeroTruncatedPoisson
117132,ENST00000618786.1,240,299,X,332.500,+,10515058-A,ZeroTruncatedPoisson


In [19]:
peaks_count = peaks_RN7SL1.groupby(['sample_id', 'method'], as_index=False)['rna_id'].count()
peaks_count.head()

,sample_id,method,rna_id
0,10515058-A,ZeroTruncatedNegativeBinomial,2
1,10515058-A,ZeroTruncatedNegativeBinomialRegression,1
2,10515058-A,ZeroTruncatedPoisson,2
3,10515058-A,ZeroTruncatedPoissonRegression,2
4,12402567-A,ZeroTruncatedNegativeBinomial,1


In [22]:
peaks_count.groupby('method')['rna_id'].mean()

method
ZeroTruncatedNegativeBinomial              2.000000
ZeroTruncatedNegativeBinomialRegression    1.304348
ZeroTruncatedPoisson                       1.836066
ZeroTruncatedPoissonRegression             1.868852
Name: rna_id, dtype: float64